In [24]:
import time
import numpy as np
from matplotlib import pyplot as plt
from keras.utils import np_utils
import keras.callbacks as cb
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import RMSprop, SGD, Adam, Nadam
from keras.datasets import mnist
from keras.layers import Merge
from keras.regularizers import l1, l2, activity_l2
from keras.layers.convolutional import Convolution1D, MaxPooling1D
import random
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.decomposition import PCA
import scipy.stats as stats
import seaborn as sea
import plotly.plotly as py

In [25]:
rawdata = pd.read_csv('GJR_sequences.csv', header=0)
rawdata.head()

,name,sequence,full_sequence,ec50_t,ec50_95ci_t,ec50_95ci_lbound_t,ec50_95ci_ubound_t,ec50_c,ec50_95ci_c,ec50_95ci_lbound_c,ec50_95ci_ubound_c,ec50_pred_t,ec50_pred_c,ec50_rise_t,ec50_rise_t.1,min_ec50_rise,max_ec50_95ci
0,EEHEE_0001.pdb,GSQEVNSGTQTYKNASPEEAERIARKAGATTWTEKGNKWEIRI,GGGSASHMGSQEVNSGTQTYKNASPEEAERIARKAGATTWTEKGNK...,1.902499,0.2,1.8,2.0,0.927242,0.4,0.7,1.1,1.167213,0.805684,0.350821,0.350821,0.057998,0.4
1,EEHEE_0001.pdb_hp,GSTTIEEAQNKKYQAEPRSWTKAGRTIGGKNWETEVNRAEASI,GGGSASHMGSTTIEEAQNKKYQAEPRSWTKAGRTIGGKNWETEVNR...,1.213600,5.5,0.8,6.3,1.272084,6.0,0.8,6.8,2.385720,1.898771,-0.559243,-0.559243,-0.559243,6.0
2,EEHEE_0001.pdb_random,GSRETKKITTVGARGEATAEQAATEEGPKNNSRISNYKEQWWI,GGGSASHMGSRETKKITTVGARGEATAEQAATEEGPKNNSRISNYK...,3.906570,0.2,3.8,4.0,4.357322,5.0,2.4,7.4,2.748009,1.442123,0.552774,0.552774,0.552774,5.0
3,EEHEE_0002.pdb,GSYELEVGNYRYRADDPEQLKEEAKKVGARDVQTDGNNFKVRG,GGGSASHMGSYELEVGNYRYRADDPEQLKEEAKKVGARDVQTDGNN...,0.178086,1.2,-0.5,0.7,1.084055,1.1,0.5,1.6,0.665333,-0.298336,-0.232476,-0.232476,-0.232476,1.2
4,EEHEE_0002.pdb_hp,GSYKGDLLYENREADNVYKATKRGRDPFGERGKEDVQNVEVQA,GGGSASHMGSYKGDLLYENREADNVYKATKRGRDPFGERGKEDVQN...,1.656384,0.7,1.4,2.1,0.064134,0.9,-0.5,0.4,1.413589,0.862263,0.115843,0.115843,-0.380805,0.9


In [26]:
rawdata.shape

(48608, 17)

In [27]:
good_data = rawdata.loc[rawdata['max_ec50_95ci'] < 1]

In [28]:
good_data.shape

(39845, 17)

In [29]:
seqlengthHash = {}

seqList = good_data['full_sequence'].tolist()
for s in seqList:
    if len(s) in seqlengthHash:
        seqlengthHash[len(s)] = seqlengthHash[len(s)] + 1
    else:
        seqlengthHash[len(s)] = 1

print "Hash of sequence lengths: ", seqlengthHash

Hash of sequence lengths:  {66: 39845}


In [30]:
#df['sequence'] = good_data[['full_sequence']]
#df = pd.DataFrame([good_data[['full_sequence']], good_data[['min_ec50_rise']]], columns=['sequence', 'output1'])

df = good_data[['full_sequence', 'min_ec50_rise']].copy()

In [31]:
df.head()

,full_sequence,min_ec50_rise
0,GGGSASHMGSQEVNSGTQTYKNASPEEAERIARKAGATTWTEKGNK...,0.057998
4,GGGSASHMGSYKGDLLYENREADNVYKATKRGRDPFGERGKEDVQN...,-0.380805
5,GGGSASHMGSYAKDKEGLYDAGYGTRRPEVADRGNEEKVKLNVNEQ...,-0.105271
6,GGGSASHMGSWRVHFRGETYTADTEDDAKQLAKDAGARRIESSNGE...,0.052325
8,GGGSASHMGSEELTHNWDEEFAGVRQTGDSATRYGVAKAVKERDLA...,-0.752859


In [32]:
df = df.reset_index()

In [33]:
df.head()

,index,full_sequence,min_ec50_rise
0,0,GGGSASHMGSQEVNSGTQTYKNASPEEAERIARKAGATTWTEKGNK...,0.057998
1,4,GGGSASHMGSYKGDLLYENREADNVYKATKRGRDPFGERGKEDVQN...,-0.380805
2,5,GGGSASHMGSYAKDKEGLYDAGYGTRRPEVADRGNEEKVKLNVNEQ...,-0.105271
3,6,GGGSASHMGSWRVHFRGETYTADTEDDAKQLAKDAGARRIESSNGE...,0.052325
4,8,GGGSASHMGSEELTHNWDEEFAGVRQTGDSATRYGVAKAVKERDLA...,-0.752859


In [34]:
df = df.drop('index', axis=1)

In [35]:
df.head()

,full_sequence,min_ec50_rise
0,GGGSASHMGSQEVNSGTQTYKNASPEEAERIARKAGATTWTEKGNK...,0.057998
1,GGGSASHMGSYKGDLLYENREADNVYKATKRGRDPFGERGKEDVQN...,-0.380805
2,GGGSASHMGSYAKDKEGLYDAGYGTRRPEVADRGNEEKVKLNVNEQ...,-0.105271
3,GGGSASHMGSWRVHFRGETYTADTEDDAKQLAKDAGARRIESSNGE...,0.052325
4,GGGSASHMGSEELTHNWDEEFAGVRQTGDSATRYGVAKAVKERDLA...,-0.752859


In [36]:
df.shape

(39845, 2)

In [38]:
from pandas import ExcelWriter
writer = ExcelWriter('GJR_cleaned_seq_only.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()